# Processing the results of assited labeling

The assited labeling notebook yields several csv files one for each query. As the process have been parallelized, we have as many copies as threads we have run. So there is the need for merging the results in one single excel file per policy instrument.

In [8]:
import boto3
import codecs
import csv
import json
import numpy as np
import pandas as pd

## Functions

In [11]:
def aws_credentials(path, filename):
    file = path + filename
    with open(file, 'r') as dict:
        key_dict = json.load(dict)
    for key in key_dict:
        KEY = key
        SECRET = key_dict[key]
    return KEY, SECRET

def read_csv_from_s3(s3_object, columns):
    for row in csv.DictReader(codecs.getreader("utf-8")(s3_object["Body"])):
        for column in columns:
            print(row[column])

## Connecting to S3

In [7]:
path = "C:/Users/jordi/Documents/claus/"
filename = "AWS_S3_keys_wri.json"
aws_id, aws_secret = aws_credentials(path, filename)
region = 'us-east-1'

bucket = 'wri-nlp-policy'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = aws_id,
    aws_secret_access_key = aws_secret
)

## Loading the policy instruments

In [4]:
# Define queries
path = "../../input/"
filename = "English_queries.xlsx"
file = path + filename
df = pd.read_excel(file, engine='openpyxl', sheet_name = "Hoja1", usecols = "A:C")

policy_intrument = {}
for index, row in df.iterrows():
    policy_intrument[row['Policy instrument']] = 0

## Loading csv files

In [13]:

for i, obj in enumerate(s3.Bucket(bucket).objects.all().filter(Prefix="english_documents/assisted_labeling/")):
    if i < 1:
        read_csv_from_s3(obj, [1, 2])

TypeError: 's3.ObjectSummary' object is not subscriptable